<a href="https://colab.research.google.com/github/Mastermind305/TOMATO-LEAF-DISEASE-CLASSIFICATION-BY-DEEP-LEARNING/blob/main/tomato_ResNet_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Models
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unrar x '/content/drive/MyDrive/tomato/train.rar'
!unrar x '/content/drive/MyDrive/tomato/val.rar'

In [ ]:
import os
folder_path1 = '/content/train'
folder_path2 = '/content/val'
for dirname, _, filenames in os.walk(folder_path1):
    print(dirname)

In [ ]:
print('Total Bacterial images:', len(os.listdir
(folder_path1 + '/Tomato___Bacterial_spot')))
print('Total Early blight images:', len(os.listdir(folder_path1 + '/Tomato___Early_blight')))
print('Total Late blight images:', len(os.listdir(folder_path1 + '/Tomato___Late_blight')))
print('Total Leaf_mold images:', len(os.listdir(folder_path1 + '/Tomato___Leaf_Mold')))
print('Total Septoria leaf images:', len(os.listdir(folder_path1 + '/Tomato___Septoria_leaf_spot')))
print('Total spider mites images:', len(os.listdir(folder_path1 + '/Tomato___Spider_mites Two-spotted_spider_mite')))
print('Total target spot images:', len(os.listdir(folder_path1 + '/Tomato___Target_Spot')))
print('Total yellow sopt images:', len(os.listdir(folder_path1 + '/Tomato___Tomato_Yellow_Leaf_Curl_Virus')))
print('Total mose:', len(os.listdir(folder_path1 + '/Tomato___Tomato_mosaic_virus')))
print('Total healthy:', len(os.listdir(folder_path1 + '/Tomato___healthy')))

# /content/tomato/train/Tomato___Early_blight


In [7]:
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
import os
for i in os.listdir(folder_path1):
  # print(i)
  k= 0
  for j in os.listdir(folder_path1 +"/"+ i):
    os.rename(folder_path1 + "/"+i + "/" + j, folder_path1 +"/"+i+"/"+ str(k)+".JPG" )
    k += 1
    print(j)

In [ ]:
from tensorflow.keras.preprocessing import image
fig = plt.figure(figsize=(18, 16))

# setting values to rows and column variables
rows = 2
columns = 2

# reading images
Image1 = image.load_img(folder_path1 + '/Tomato___Bacterial_spot/0.JPG')
Image2 = image.load_img(folder_path1 + '/Tomato___Early_blight/101.JPG')
Image3 = image.load_img(folder_path1 + '/Tomato___Septoria_leaf_spot/108.JPG')
Image4 = image.load_img(folder_path1 + '/Tomato___Tomato_mosaic_virus/142.JPG')

fig.add_subplot(rows, columns, 1)
plt.imshow(Image1)
plt.title("First")

fig.add_subplot(rows, columns, 2)
plt.imshow(Image2)
plt.title("Second")

fig.add_subplot(rows, columns, 3)
plt.imshow(Image3)
plt.title("Third")

fig.add_subplot(rows, columns, 4)
plt.imshow(Image4)
plt.title("Fourth")

In [10]:
train_datagen = ImageDataGenerator(
                fill_mode = 'nearest',
                validation_split=0.2
)

In [ ]:
train_generator=train_datagen.flow_from_directory(
    folder_path1,
    target_size=(108,108),
    color_mode='rgb',
    class_mode='categorical',
    subset='training',
)
validation_generator=train_datagen.flow_from_directory(
    folder_path2,
    target_size=(108,108),
    color_mode='rgb',
    class_mode='categorical',
    subset='validation',
)


In [ ]:
from keras.applications import ResNet50
model = tf.keras.models.Sequential([
    ResNet50(input_shape=(108,108,3), include_top=False),
])
for layer in model.layers:
  layer.trainable = False

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))

model.summary()

In [13]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
x = model.fit_generator(train_generator,
                    validation_data=validation_generator,
                    epochs=20,
                    verbose=1,
                    steps_per_epoch=len(train_generator),
                    validation_steps=len(validation_generator,)
                    )

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x.history['accuracy'])
plt.plot(x.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:
plt.plot(x.history['loss'])
plt.plot(x.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:
model.save('tomato_leaf_resnet_50.h5')